In [43]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

In [ ]:
y ~ NormalMeanVariance(mean, var) where { q = q(μ)q(v)q(out)  }

In [60]:
@model function smoothing(n_observations, noise_mean, noise_var)
    
   x_prior_mean = datavar(Float64)
   x_prior_var  = datavar(Float64)
   
   x_prior ~ NormalMeanVariance(x_prior_mean, x_prior_var)
   
   x = randomvar(n_observations)
   y = datavar(Float64, n_observations)
   
   x[1] = x_prior
   
   for i in 2:n_observations
       x[i] ~ x[i - 1] + 1.0
       y[i] ~ x[i] + NormalMeanVariance(noise_mean, noise_var)
   end
   
   return x_prior_mean, x_prior_var, x, y
end

quote
    #= /Users/bvdmitri/.julia/dev/GraphPPL.jl/src/GraphPPL.jl:115 =#
    function smoothing(n_observations, noise_mean, noise_var)
        #= /Users/bvdmitri/.julia/dev/GraphPPL.jl/src/GraphPPL.jl:115 =#
        #= /Users/bvdmitri/.julia/dev/GraphPPL.jl/src/GraphPPL.jl:116 =#
        var"##model#391" = Model()
        #= /Users/bvdmitri/.julia/dev/GraphPPL.jl/src/GraphPPL.jl:117 =#
        begin
            #= In[60]:1 =#
            #= In[60]:3 =#
            x_prior_mean = ReactiveMP.datavar(var"##model#391", :x_prior_mean, Dirac{Float64})
            #= In[60]:4 =#
            x_prior_var = ReactiveMP.datavar(var"##model#391", :x_prior_var, Dirac{Float64})
            #= In[60]:6 =#
            (var"##392", x_prior = ReactiveMP.make_node(var"##model#391", NormalMeanVariance, ReactiveMP.AutoVar(:x_prior), ReactiveMP.as_variable(var"##model#391", Symbol("##arg#393"), x_prior_mean), ReactiveMP.as_variable(var"##model#391", Symbol("##arg#394"), x_prior_var)))
            #= In[60]

In [53]:
noise_real_mean = 0.0
noise_real_var  = 10.0

n = 500
data = map(d -> d + rand(Normal(noise_real_mean, sqrt(noise_real_var))), collect(1:n));

In [54]:
function inference(; data, x_prior)
    n = length(data)
    
    x_prior_mean, x_prior_var, x, y = smoothing(n, noise_real_mean, noise_real_var);

    subscriptions = Vector{Teardown}(undef, n)
    marginals = Vector{Marginal}(undef, n)
    
    for i in 1:n
         @inbounds subscriptions[i] = subscribe!(getmarginal(x[i]), (m) -> marginals[i] = m)
    end
    
    update!(x_prior_mean, mean(x_prior))
    update!(x_prior_var, var(x_prior))
    
    for i in 1:n
        @inbounds update!(y[i], data[i])
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return marginals
end

inference (generic function with 1 method)

In [61]:
inference(
    data = data,
    x_prior = NormalMeanVariance(0.0, 10000.0)
)

LoadError: UndefVarError: ##382 not defined